In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import datetime
import json
from sqlalchemy.types import BigInteger, Time, Date, Float, Integer, Boolean, Text, TIMESTAMP

In [2]:
import sys

In [3]:
sys.path.append('python_scripts/ETL_python_scripts/')

In [4]:
import get_weather_from_google_search
import get_data_from_api
import ETL_utilities
import db_connection

In [5]:
city_names = ['Moscow', 'London']
columns = ['weather_id',
           'date',
           'time',
           'city_name',
           'temp_c',
           'temp_f']
db_df = pd.DataFrame(columns=columns)

In [6]:
#take a city
for city in city_names:
    #put city in question
    question = f'weather {city}'
    #make soup with question
    soup = get_data_from_api.soup_from_google_search(question)
    #make json from soup
    output_json = \
    get_weather_from_google_search.google_soup_to_json_weather(soup)
    #create a dictionary to save the results
    data_dict = {
    #create unique id
    'weather_id': ETL_utilities.create_unique_id_from_date(),
    #create date feature
    'date': datetime.now().date(),
    'time': datetime.now().time(),
    #data from json
    #load string json as dict and write in data dict
    'city_name': city,
    'temp_c': json.loads(output_json)['temperature_c'],
    'temp_f': json.loads(output_json)['temperature_f']}
    #convert dictionary to df for concat
    data_dict_df = pd.DataFrame([data_dict])
    db_df = pd.concat([db_df,data_dict_df])

In [7]:
db_df

,weather_id,date,time,city_name,temp_c,temp_f
0,202301-3109-5339-4403354e-a822-4f3e-8809-e5a94...,2023-01-31,09:53:39.299037,Moscow,0.0,32.0
0,202301-3109-5340-dd31d6e5-eb40-4480-af43-d6399...,2023-01-31,09:53:40.591383,London,8.0,47.0


In [8]:
engine = db_connection.engine
weather_dtypes = {'time': Time, 
        'date': Date, 
        'temp_c': Float, 
        'temp_f': Float}

In [9]:
weather_sql_query = '''
CREATE TABLE IF NOT EXISTS weather_prod(
weather_id TEXT,
date DATE,
time TIME,
city_name TEXT,
temp_c FLOAT,
temp_f FLOAT);

INSERT INTO weather_prod (weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f)
(select weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f
from weather_tmp);
    '''

In [10]:
db_connection.load_df_to_tpm_table(db_df, 'weather',  weather_dtypes)

weather loading into DB...
weather_tmp loaded


In [11]:
db_connection.load_tmp_table_to_prod_table(weather_sql_query)